In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from pandas_datareader import data
from scipy.stats import norm

%matplotlib inline

In [2]:
!pip install tabulate
from tabulate import tabulate

In [3]:
import csv
df = pd.read_csv('Final_stock_data_V2.csv')
df

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149,0.355467,0.049267
1,ABT,Abbott Laboratories,Healthcare,Increase,Above Fair Price,OverValued,Average,118.2900,0.181179,2.091710e+11,...,0.74,3.940,0.1318,22.924420,30.022842,51.234756,25.96,0.109154,0.249769,0.247649
2,ACN,Accenture plc,Technology,Increase,Above Fair Price,OverValued,Sustainable,323.9050,0.208836,2.047083e+11,...,1.21,9.613,0.1135,31.025385,33.694477,121.148794,9.45,0.166287,0.254874,0.245322
3,ADM,Archer-Daniels-Midland Company,Consumer Defensive,Increase,Above Fair Price,OverValued,Not Sustainable,76.9850,0.122815,4.327843e+10,...,0.81,4.790,0.0660,14.804809,16.072025,19.449588,36.42,0.117663,0.252289,0.150693
4,ADP,"Automatic Data Processing, Inc.",Industrials,Increase,Above Fair Price,OverValued,Sustainable,201.0100,0.182708,8.443324e+10,...,0.83,6.460,0.1371,26.483530,31.116098,90.801919,14.18,0.120094,0.272415,0.178939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,...,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929,0.405374,-0.032851
934,ZD,"Ziff Davis, Inc.",Communication Services,Increase,Above Fair Price,OverValued,No Data,102.4800,0.145864,4.941668e+09,...,1.01,4.120,0.1140,14.193907,24.873789,38.485339,999.00,0.141975,0.308007,0.055993
935,ZUMZ,Zumiez Inc.,Consumer Cyclical,Decrease,Above Fair Price,UnderValued,No Data,42.6600,0.035214,9.777245e+08,...,1.68,4.870,0.1500,9.233767,8.759754,21.330001,999.00,0.223419,0.513665,0.157523
936,ESNT,Essent Group Ltd.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,44.4800,0.082041,4.912015e+09,...,1.33,5.569,0.1555,7.339934,7.987071,23.215807,999.00,0.180874,0.502764,0.045034


In [4]:
#using only undervalued stocks for our portfolio models
df2 = df.loc[df['capm_status'] == 'UnderValued']
df2

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149,0.355467,0.049267
5,ADSK,"Autodesk, Inc.",Technology,Increase,Below Fair Price,UnderValued,Sustainable,217.3800,0.190997,4.781773e+10,...,1.43,5.928,0.2883,31.967648,36.670040,302.038344,15.45,0.193030,0.384894,0.209161
8,AFL,AFLAC Incorporated,Financial Services,Decrease,Above Fair Price,UnderValued,Sustainable,63.0500,0.131485,4.111692e+10,...,0.93,6.390,0.0316,11.484518,9.866980,11.857047,17.30,0.132250,0.300487,0.145033
11,ALK,"Alaska Air Group, Inc.",Industrials,Increase,Above Fair Price,UnderValued,No Data,58.9750,0.141697,7.425660e+09,...,1.55,3.770,-0.2340,9.073076,15.643236,0.761055,999.00,0.207617,0.454054,-0.086043
12,ALLE,Allegion plc,Industrials,Increase,Above Fair Price,UnderValued,Average,114.1447,0.097180,1.023831e+10,...,1.14,5.086,0.0777,19.955368,22.442923,31.814578,23.59,0.157777,0.279213,0.118004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,XOMA,XOMA Corporation,Healthcare,Increase,Above Fair Price,UnderValued,No Data,20.7600,-0.034853,2.348205e+08,...,0.81,0.082,0.1900,-90.260870,253.170730,14.120154,999.00,0.117663,0.689078,0.371572
932,XP,XP Inc.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,34.1100,-0.001020,1.908622e+10,...,1.91,0.986,0.0478,27.508064,34.594322,7.380346,999.00,0.251378,NaN,NaN
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,...,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929,0.405374,-0.032851
935,ZUMZ,Zumiez Inc.,Consumer Cyclical,Decrease,Above Fair Price,UnderValued,No Data,42.6600,0.035214,9.777245e+08,...,1.68,4.870,0.1500,9.233767,8.759754,21.330001,999.00,0.223419,0.513665,0.157523


In [8]:
#to meet 2/3 criteria to be selected for portfolio model
df3 = df2.loc[(df2['Earnings_expectation'] == 'Increase') | (df2['irv_status'] == 'Below Fair Price')]
df3

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
0,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,214.1500,0.103398,1.308405e+10,...,1.25,9.550,0.1525,13.932986,22.424082,109.188247,12.97,0.171149,0.355467,0.049267
5,ADSK,"Autodesk, Inc.",Technology,Increase,Below Fair Price,UnderValued,Sustainable,217.3800,0.190997,4.781773e+10,...,1.43,5.928,0.2883,31.967648,36.670040,302.038344,15.45,0.193030,0.384894,0.209161
11,ALK,"Alaska Air Group, Inc.",Industrials,Increase,Above Fair Price,UnderValued,No Data,58.9750,0.141697,7.425660e+09,...,1.55,3.770,-0.2340,9.073076,15.643236,0.761055,999.00,0.207617,0.454054,-0.086043
12,ALLE,Allegion plc,Industrials,Increase,Above Fair Price,UnderValued,Average,114.1447,0.097180,1.023831e+10,...,1.14,5.086,0.0777,19.955368,22.442923,31.814578,23.59,0.157777,0.279213,0.118004
15,AME,"AMETEK, Inc.",Industrials,Increase,Above Fair Price,UnderValued,Average,128.2800,0.155568,2.967437e+10,...,1.31,3.990,-0.0120,23.799630,32.150375,16.355389,25.95,0.178443,0.269622,0.206944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,WWD,"Woodward, Inc.",Industrials,Increase,Above Fair Price,UnderValued,No Data,120.0800,0.111932,7.573710e+09,...,1.64,3.180,0.1820,25.333336,37.761005,76.863011,999.00,0.218557,0.389052,0.119014
931,XOMA,XOMA Corporation,Healthcare,Increase,Above Fair Price,UnderValued,No Data,20.7600,-0.034853,2.348205e+08,...,0.81,0.082,0.1900,-90.260870,253.170730,14.120154,999.00,0.117663,0.689078,0.371572
932,XP,XP Inc.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,34.1100,-0.001020,1.908622e+10,...,1.91,0.986,0.0478,27.508064,34.594322,7.380346,999.00,0.251378,NaN,NaN
933,XRX,Xerox Holdings Corporation,Technology,Increase,Above Fair Price,UnderValued,No Data,20.6059,0.029031,3.678401e+09,...,1.75,1.459,-0.1600,10.302950,14.123303,0.609810,999.00,0.231929,0.405374,-0.032851


In [9]:
#getting df for each sector
df_basic_materials = df3.loc[df['sector'] == 'Basic Materials']
df_communication = df3.loc[df['sector'] == 'Communication Services']
df_consumer_c = df3.loc[df['sector'] == 'Consumer Cyclical']
df_consumer_d = df3.loc[df['sector'] == 'Consumer Defensive']
df_energy = df3.loc[df['sector'] == 'Energy']
df_financial = df3.loc[df['sector'] == 'Financial Services']
df_healthcare = df3.loc[df['sector'] == 'Healthcare']
df_industrials = df3.loc[df['sector'] == 'Industrials']
df_real_estate = df3.loc[df['sector'] == 'Real Estate']
df_tech = df3.loc[df['sector'] == 'Technology']
df_utilites = df3.loc[df['sector'] == 'Utilities']

In [10]:
#calculating the average volatility for all sectors

basic_AHV = df_basic_materials['AHV_5y'].mean()
comm_AHV = df_communication['AHV_5y'].mean()
consumer_c_AHV = df_consumer_c['AHV_5y'].mean()
consumer_d_AHV = df_consumer_d['AHV_5y'].mean()
energy_AHV = df_energy['AHV_5y'].mean()
financial_AHV = df_financial['AHV_5y'].mean()
healthcare_AHV = df_healthcare['AHV_5y'].mean()
industrials_AHV = df_industrials['AHV_5y'].mean()
real_estate_AHV = df_real_estate['AHV_5y'].mean()
tech_AHV = df_tech['AHV_5y'].mean()
utilities_AHV = df_utilites['AHV_5y'].mean()

print(tabulate([['Basic Materials', basic_AHV],['Communication Services', comm_AHV],['Consumer Cyclical', consumer_c_AHV],['Consumer Defensive', consumer_d_AHV], ['Energy', energy_AHV], ['Financial Services', financial_AHV], ['Healthcare', healthcare_AHV], ['Industrials', industrials_AHV], ['Real Estate', real_estate_AHV], ['Technology', tech_AHV], ['Utilities', utilities_AHV]],
               headers=['Sector','Historical Volatility']))

Sector                    Historical Volatility
----------------------  -----------------------
Basic Materials                        0.514
Communication Services                 0.500482
Consumer Cyclical                      0.514444
Consumer Defensive                     0.407907
Energy                                 0.523126
Financial Services                     0.408439
Healthcare                             0.552295
Industrials                            0.45136
Real Estate                            0.406957
Technology                             0.478915
Utilities                              0.260092


From above, the 3 least volatile sectors are Utilities, Real Estate, Consumer Defensive
- returns only 24 rows

From above, the 4 least volatile sectors are Utilities, Real Estate, Consumer Defensive, Financial Services
- returns only 89 rows

In [14]:
#filter dataframe to show lowest 3 sectors
df4 = df3.loc[(df3['sector'] == 'Utilities') | (df3['sector'] == 'Real Estate') | (df3['sector'] == 'Consumer Defensive') | (df3['sector'] == 'Financial Services')]
df4

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
26,AXP,American Express Company,Financial Services,Increase,Below Fair Price,UnderValued,Not Sustainable,195.110,0.155990,1.511236e+11,...,1.24,9.573,0.4330,20.176836,20.381280,706.617956,20.34,0.169933,0.343000,0.210121
28,BEN,"Franklin Resources, Inc.",Financial Services,Increase,Above Fair Price,UnderValued,Sustainable,29.930,0.009948,1.501872e+10,...,1.20,3.570,0.0949,7.876316,8.383754,9.619805,19.51,0.165071,0.345803,-0.016066
31,BK,The Bank of New York Mellon Cor,Financial Services,Increase,Above Fair Price,UnderValued,Average,55.740,0.123563,4.603126e+10,...,1.15,3.925,0.1356,12.196937,14.201275,24.881978,22.79,0.158993,0.298729,0.053724
35,BXP,"Boston Properties, Inc.",Real Estate,Increase,Above Fair Price,UnderValued,Sustainable,121.005,0.052115,1.890183e+10,...,1.16,2.044,0.0700,39.673770,59.200100,31.618901,13.57,0.160209,0.318134,0.015426
36,CAG,"ConAgra Brands, Inc.",Consumer Defensive,Increase,Above Fair Price,UnderValued,Not Sustainable,35.700,0.090652,1.712493e+10,...,0.79,2.481,0.0224,13.522727,14.389359,6.193629,32.92,0.115232,0.287178,0.000833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,VCTR,"Victory Capital Holdings, Inc.",Financial Services,Increase,Above Fair Price,UnderValued,No Data,32.210,0.110286,2.193565e+09,...,1.08,3.551,0.1170,6.100379,9.070684,12.392493,999.00,0.150484,NaN,NaN
915,VLY,Valley National Bancorp,Financial Services,Increase,Above Fair Price,UnderValued,No Data,13.985,0.063284,5.691251e+09,...,1.21,1.110,0.0500,11.654166,12.599098,3.083582,999.00,0.166287,0.360308,0.060302
922,WAFD,"Washington Federal, Inc.",Financial Services,Increase,Above Fair Price,UnderValued,No Data,35.920,0.109876,2.342835e+09,...,0.85,2.390,0.0700,11.777049,15.029287,9.385982,999.00,0.122525,0.323025,0.026584
932,XP,XP Inc.,Financial Services,Increase,Above Fair Price,UnderValued,No Data,34.110,-0.001020,1.908622e+10,...,1.91,0.986,0.0478,27.508064,34.594322,7.380346,999.00,0.251378,NaN,NaN


In [15]:
#get annualized returns for stocks and pick the top 20-30 for the income model

df4.sort_values(by=['annu_return_10y'])
df4.head(30)

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,beta,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,ESG_risk,capm_expected_return,AHV_5y,annu_return_5y
26,AXP,American Express Company,Financial Services,Increase,Below Fair Price,UnderValued,Not Sustainable,195.1100,0.155990,1.511236e+11,...,1.24,9.573,0.4330,20.176836,20.381280,706.617956,20.34,0.169933,0.343000,0.210121
28,BEN,"Franklin Resources, Inc.",Financial Services,Increase,Above Fair Price,UnderValued,Sustainable,29.9300,0.009948,1.501872e+10,...,1.20,3.570,0.0949,7.876316,8.383754,9.619805,19.51,0.165071,0.345803,-0.016066
31,BK,The Bank of New York Mellon Cor,Financial Services,Increase,Above Fair Price,UnderValued,Average,55.7400,0.123563,4.603126e+10,...,1.15,3.925,0.1356,12.196937,14.201275,24.881978,22.79,0.158993,0.298729,0.053724
35,BXP,"Boston Properties, Inc.",Real Estate,Increase,Above Fair Price,UnderValued,Sustainable,121.0050,0.052115,1.890183e+10,...,1.16,2.044,0.0700,39.673770,59.200100,31.618901,13.57,0.160209,0.318134,0.015426
36,CAG,"ConAgra Brands, Inc.",Consumer Defensive,Increase,Above Fair Price,UnderValued,Not Sustainable,35.7000,0.090652,1.712493e+10,...,0.79,2.481,0.0224,13.522727,14.389359,6.193629,32.92,0.115232,0.287178,0.000833
39,CBRE,CBRE Group Inc,Real Estate,Increase,Above Fair Price,UnderValued,Sustainable,99.5300,0.182808,3.330930e+10,...,1.40,4.290,0.1100,17.647163,23.200466,36.186806,6.99,0.189383,0.351216,0.244065
46,CL,Colgate-Palmolive Company,Consumer Defensive,Increase,Above Fair Price,UnderValued,Average,79.0900,0.078620,6.666093e+10,...,0.62,3.140,0.0612,23.330381,25.187897,19.186076,22.12,0.094567,0.206239,0.051918
51,COF,Capital One Financial Corporati,Financial Services,Decrease,Below Fair Price,UnderValued,Average,153.6800,0.140197,6.540959e+10,...,1.65,26.739,0.4520,7.958571,5.747410,626.622996,23.22,0.219773,0.398322,0.129511
78,ETR,Entergy Corporation,Utilities,Increase,Above Fair Price,UnderValued,Not Sustainable,105.0550,0.092009,2.111406e+10,...,0.61,6.179,0.0600,16.570189,17.001942,25.226614,30.28,0.093351,0.260092,0.118782
111,IVZ,Invesco Ltd,Financial Services,Increase,Above Fair Price,UnderValued,Not Sustainable,22.0725,0.030092,1.017999e+10,...,1.23,2.530,0.2130,7.911291,8.724308,17.835618,20.28,0.168718,0.427328,-0.024774
